### Geosmart Hackweek notes

Contributors: Aminat Ambelorun, Preetika Kaur, Ziqi Yin, Steven Pestana

 - Goal: to learn about different ML algorithms, git, python, typical ANN workflow for real world studies.

 - Outcomes: (1) We learned the MLP algorithm in the National Snow Model, and related parameters (e.g., activation function, optimizer); (2) We tested the sensitivity of the model performance to several parameters (shuffle, epochs, batch_size, number of layer and nodes); (3) tried linear regression.
 
 - Increasing batch_size reduces R2 due to more overlapping; increasing epochs gives better training performance but needs more training time; linear regression gives bad performance for this task, but probably will perform better with less data.

| Optimazation method | Default | New | R2(N_Sierras) | R2(S_Sierras_Low) | R2(S_Sierras_High) |
| --- | --- | --- | --- | --- | --- | 
| Original |  |  | 0.86 | 0.98 | 0.97 |
| Shuffle | True | False | 0.90 | 0.97 | 0.97 |
| Epochs | 30 | 100 | 0.96 | 0.98 | 0.98 |
| Epochs | 30 | 50 | 0.91 | 0.98 | 0.98 |
| Epochs | 30 | 10 | 0.81 | 0.83 | 0.71 |
| Epochs | 30 | 1 | -0.32 | -0.25 | -0.03 |
| Batch_size | 100 | 50 | 0.91 | 0.98 | 0.98 |
| Batch_size | 100 | 200 | 0.84 | 0.91 | 0.93 |
| Batch_size | 100 | 500 | 0.78 | 0.36 | 0.80 |
| Linear Regression |  |  | -0.50 | -0.14 | 0.31 |

<img align = 'center' src="./Images/Superior.JPG" alt = 'image' width = '1000'/>


# Model Training


This notebook exemplifies the Sierra Snow Model (SSM) (a derivation of the National Snow Model (NSM)) data processing (through the DataProcess.py script), and model training, predictions, and preliminary evaluation via the MLP_model.py script.
With the focus of GeoSMART Hack Week to advance machine learning modeling skill sets, the motivation of the SSM project is for participants to modify the MLP_model.py script.
Suggested modifications include optimizing the current multilayered-perceptron (MLP) neural network algorithm or selecting and optimizing a different ML algorithm (strongly encouraged).
We encourage advanced modelers to contact Dr. Ryan Johnson (rjohnson18@ua.edu) for ideas and methods to optimize the model framework, there are several concepts of interest including feature engineering, domain optimization, feature selection, etc.

The following workflow exemplifies the steps and python files to process the training data, train a model, produce predictions, and perform preliminary evaluations

In [1]:
import os
import DataProcess
import MLP_Model
#Set working directories
cwd = os.getcwd()
os.chdir("..")
os.chdir("..")
datapath = os.getcwd()  

2023-10-27 03:41:38.551974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-27 03:41:38.552007: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Model Training and Testing Schema

The motivation the project is to advance the SSM skill for extrapolating regional SWE dynamics from in-situ observations.
To develop and test the SSM, we will train the model on NASA Airborne Snow Observatory (ASO) and snow course observations spanning 2013-2018, and some of 2019.
Within this training dataset, model training will use a random 75-25\% train-test data split. 
The random sample function will be 1234 to ensure all participants models use the same training and testing data for this phase of model development - note, this will support an intermodel comparision.

Model validation will be on water year 2019 and use the [NWM_MLP_2019_Simulation]('./NWM_MLP_2019_Simulation.ipynb').
This historical simulation will function as a hindcast, and use the 2019 water year NASA ASO and snow course observations to determine model performance. 


Upon the completion of model training, model execution predicts 1-km resolution SWE from data up to the current date of observation provided Latitude, Longitude, corresponding topographic data, and neighboring observation input features. From the sampling of test features, Chapter [Evaluation]('./evaluation.ipynb') compares the modeled 1-km grid SWE values to the observed values.

In [2]:
#Define hold out year
HOY = 2019
#Run data processing script to partition key regional dataframes
#note, need to load RegionTrain_SCA.h5,
RegionTrain, RegionTest, RegionObs_Train, RegionObs_Test, RegionTest_notScaled = DataProcess.DataProcess(HOY, datapath, cwd)

Processing training dataframes for each region
N_Sierras
S_Sierras_Low
S_Sierras_High


In [3]:
RegionTrain['N_Sierras']

,SWE_CDEC__DSS,SWE_SNOTEL__784_CA_SNTL,Prev_SWE_CDEC__MRL,SWE_SNOTEL__1242_NV_SNTL,Prev_SWE_SNOTEL__1258_CA_SNTL,SWE_CDEC__BLA,Long,SWE_SNOTEL__652_NV_SNTL,SWE_CDEC__SLT,SWE_SNOTEL__848_CA_SNTL,...,Delta_SWE_CDEC__BKL,Delta_SWE_SNOTEL__541_CA_SNTL,Delta_SWE_CDEC__SLT,Delta_SWE_CDEC__ADM,Delta_SWE_CDEC__CSL,Delta_SWE_CDEC__HIG,Delta_SWE_CDEC__GKS,Date,VIIRS_SCA,hasSnow
index,,,,,,,,,,,,,,,,,,,,,
76b55900-eb3d-4d25-a538-f74302ffe72d,0.464345,0.386861,0.383987,0.000000,0.000000,0.720062,1.000000,0.317938,0.430608,0.265152,...,0.638514,0.421818,0.572254,0.773723,0.544118,0.567686,0.389908,2013-02-19,100.00,True
147d5eb4-e574-47e4-994a-8a2908c06050,0.227197,0.051095,0.223856,0.321990,0.000000,0.421462,0.602041,0.024705,0.189422,0.048485,...,0.604730,0.436364,0.572254,0.729927,0.485294,0.576419,0.408257,2014-01-07,100.00,True
6dd37e24-0a9c-4749-8ab6-7e26d42925d6,0.240464,0.142336,0.197712,0.261780,0.478873,0.416796,0.086735,0.118153,0.187547,0.050000,...,0.614865,0.436364,0.572254,0.759124,0.612745,0.611354,0.394495,2017-12-19,100.00,True
df01bf44-46b8-4541-b0a4-ed1fae16ac38,0.308458,0.164234,0.204248,0.272251,0.469484,0.416796,0.352041,0.150376,0.201050,0.086364,...,0.655405,0.490909,0.606936,0.810219,0.558824,0.641921,0.440367,2018-01-23,7.25,False
00c4db22-a423-41a4-ada6-a8b1b04153a4,1.000000,0.920925,0.946078,0.261780,0.469484,0.780715,0.000000,0.947368,0.765941,0.822727,...,0.540541,0.414545,0.080925,0.335766,0.225490,0.000000,0.151376,2017-05-02,100.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
018cf1a1-f945-4097-9c47-0c4690538bb5,0.398010,0.221411,0.370915,0.261780,0.000000,0.463453,0.750000,0.150376,0.193923,0.083333,...,0.577703,0.429091,0.572254,0.729927,0.406863,0.593886,0.399083,2014-03-25,100.00,True
00c4db22-a423-41a4-ada6-a8b1b04153a4,0.165837,0.000000,0.163399,0.261780,0.000000,0.393468,0.000000,0.000000,0.191673,0.000000,...,0.625000,0.436364,0.578035,0.729927,0.544118,0.593886,0.403670,2014-06-24,100.00,True
018cf1a1-f945-4097-9c47-0c4690538bb5,0.915423,0.959854,0.942810,0.664921,0.563380,0.944012,0.750000,0.899033,0.887472,0.960606,...,0.618243,0.461818,0.589595,0.613139,0.446078,0.524017,0.325688,2017-04-04,54.50,True


## Multilayered Precepton Network (MLP)
Given the identified optimal feature sets using recursive feature elimination (RFE), this step trains your model.
Here, the model is an ANN multilayer perceptron (MLP) regression model to estimate SWE found in the [MLP_Model]('./MLP_Model.py') file.
This file serves as a template for Hackweek participants to modify and by following the template, participants will be able to streamline model development and evaluation.

<img align = 'left' src="./Images/MLP_architecture.png" alt = 'image' width = '450'/>

The MLP is a classical type of feedforward ANN, successfully and frequently applied in environmental modeling applications.
The MLP regression model estimates a target variable by learning a non-linear function to describe the target from an input vector of features.
It performs learning via a back-propagation algorithm over a series of hidden layers containing interconnected nodes (neurons). 
The neurons connect bordering layers by a summation of weights and an activation function transforms model outputs to predicted values (i.e., SWE (in)). 
The model calculates error and adjusts the weights to minimize the error during model training, supporting the use of 
MLPs to effectively describe a target variable with any function, continuous or discontinuous. 

Model training leveraged the Keras API within the TensorFlow library.
We selected the popular, open-source TensorFlow and Keras platforms for their wide applicability and capabilities in deep learning.
The MLP model for the region consists of an input layer containing nodes for each feature in the optimized feature space, 7 dense hidden layers, and an output layer of the target SWE value. 
Model formulation uses the Rectified Linear Unit (ReLu) activation function to transform each hidden layer to non-linearize the solution.

In [ ]:
#model training, each participants model will be different but should follow the prescribed input feature template
epochs= 30
batch_size = 100
MLP_Model.Model_train(cwd, epochs, batch_size, RegionTrain, RegionTest, RegionObs_Train, RegionObs_Test)

## Make predictions on the random sample of testing data
<img align = 'center' src="./Images/predictivemodeling.jpg" alt = 'image' width = '600'/>

The next phase of model development is to examine model performance on the random sample of testing data.
Refining model predictions at this phase will ensure the best model performance for the Hold-Out-Year validation set.

In [ ]:
#Need to create Predictions folder if running for the first time
Predictions = MLP_Model.Model_predict(cwd,  RegionTest, RegionObs_Test, RegionTest_notScaled)

In [ ]:
#Predictions['N_Sierras']

## Perform Preliminary Model Evaluation

How does your model performance? 
We are using simple model evaluation metrics of R2 and RMSE to guage model performance.
You will perform a more exhaustive model evaluation in the [Evaluation]('./evaluation.ipynb') chapter.

In [ ]:
Performance = MLP_Model.Prelim_Eval(cwd, Predictions)
Performance

### Model Evaluation

Now that we have a trained model producing acceptable performance, it is time to more rigorously evaluate its performance using the [Standardized Snow Water Equivalent Tool](./SSWEET.py) within an interactive [evaluation notebook](./evaluation.ipynb).